# Detecting and Mitigating Unfairness in Models

Machine learning models can incorporate unintentional bias, which can lead to issues with *fairness*. For example, a model that predicts the likelihood of diabetes might work well for some age groups, but not for others - subjecting a subset of patients to unnecessary tests, or depriving them of tests that would confirm a diabetes diagnosis.

In this exercise, you'll use the **Fairlearn** package to analyze a model and find any disparity in prediction performance for different subsets of patients based on age.

## Before You Start

Before you start this lab, ensure that you have Completed the *Create an Azure Machine Learning Workspace* and *Create a Compute Instance* tasks in [Getting Started with Azure Machine Learning](./labdocs/Lab01.md).

Additionally, the Fairlearn package used in this exercise has dependencies on specific versions of common Python packages. To avoid potential conflicts, you're going to create a separate Conda environment and associated Jupyter kernel specifically for this exercise. If you have not already done so, <u>close this notebook</u> and use the instructions in [Analyzing and Mitigating Unfairness in Models](./labdocs/Lab11.md) to set up the Conda environment. Then reopen this notebook and connect it to the **aml-fair** kernel to continue.

## Train a model

You'll start by training a classification model to predict the likelihood of diabetes. In addition to splitting the data into training a test sets of features and labels, you'll extract *sensitive* features that are used to define subpopulations of the data for which you want to compare fairness. In this case, you'll use the **Age** column to define two categories of patient: those over 50 years old, and those 50 or younger.

In [1]:
import os
import pandas as pd
import numpy as np
from azureml.contrib import fairness
from azureml.core import Workspace, Model, Experiment
from azureml.widgets import RunDetails
from fairlearn.metrics import _group_metric_set
from fairlearn.reductions import GridSearch, EqualizedOdds
from fairlearn.widget import FairlearnDashboard
from sklearn import model_selection, metrics
from sklearn.tree import DecisionTreeClassifier
import fairlearn.metrics
import joblib


In [2]:
print("Loading Data...")
data: pd.DataFrame = pd.read_csv('data/diabetes.csv')

features = [
    'Pregnancies', 'PlasmaGlucose', 'DiastolicBloodPressure',
    'TricepsThickness', 'SerumInsulin', 'BMI', 'DiabetesPedigree', 'Age'
]
X = data[features].to_numpy()
y = data['Diabetic'].to_numpy()

S = data[['Age']].astype(int)
S['Age'] = np.where(S.Age > 50, 'Over 50', '50 or younger')

X_train, X_test, y_train, y_test, S_train, S_test = (
    model_selection.train_test_split(
        X, y, S, test_size=0.20, random_state=0, stratify=y,
    )
)

print("Training model...")
diabetes_model = DecisionTreeClassifier()
diabetes_model.fit(X_train, y_train)

print("Model trained.")


Loading Data...
Training model...
Model trained.


Now that you've trained a model, you can use the Fairlearn package to compare its behavior for different sensitive feature values. In this case, you'll:

- Use the **selection_rate_group_summary** function to return the selection rate (percentage of positive predictions) for the overall population and for each age group in the **Age** sensitive feature.
- Use the **group_summary** function to calculate prediction performance based on three commomly used classification metrics (*accuracy*, *precision*, and *recall*) for the overall population and for each age group in the **Age** sensitive feature. Note that **scikit-learn** metric scores are used to calculate the performance values.

In [3]:
y_hat = diabetes_model.predict(X_test)

selection_rates = fairlearn.metrics.selection_rate_group_summary(
    y_test, y_hat, sensitive_features=S_test['Age'],
)
print("Selection Rates\n",selection_rates)

accuracy_scores = fairlearn.metrics.group_summary(
    metrics.accuracy_score, y_test, y_hat, sensitive_features=S_test['Age'],
)
print("\nAccuracy\n",accuracy_scores)

precision_scores = fairlearn.metrics.group_summary(
    metrics.precision_score, y_test, y_hat, sensitive_features=S_test['Age'],
)
print("\nPrecision\n",precision_scores)

recall_scores = fairlearn.metrics.group_summary(
    metrics.recall_score, y_test, y_hat, sensitive_features=S_test['Age'],
)
print("\nRecall\n",recall_scores)


Selection Rates
 {'overall': 0.3385, 'by_group': {'50 or younger': 0.3003865267807841, 'Over 50': 0.7037037037037037}}

Accuracy
 {'overall': 0.896, 'by_group': {'50 or younger': 0.8972943125345113, 'Over 50': 0.8835978835978836}}

Precision
 {'overall': 0.8404726735598228, 'by_group': {'50 or younger': 0.8253676470588235, 'Over 50': 0.9022556390977443}}

Recall
 {'overall': 0.8505231689088192, 'by_group': {'50 or younger': 0.8314814814814815, 'Over 50': 0.9302325581395349}}


From these metrics, you should be able to discern that a larger proportion of the older patients are predicted to be diabetic. *Accuracy* should be more or less equal for the two groups, but a closer inspection of *precision* and *recall* indicates some disparity in how well the model predicts for each age group.

In this scenario, consider *recall*. This metric indicates the proportion of positive cases that were correctly identified by the model. In other words, of all the patients who are actually diabetic, how many did the model find? The model does a better job of this for patients in the older age group than for younger patients.

It's often easier to compare metrics visually. To do this, you'll use the Fairlearn dashboard:

1. Run the cell below.
2. When the widget is displayed, use the **Get started** link to start configuring your visualization.
3. Select the sensitive features you want to compare (in this case, there's only one: **Age**).
4. Select the model performance metric you want to compare (in this case, it's a binary classification model so the options are *Accuracy*, *Balanced accuracy*, *Precision*, and *Recall*). Start with **Recall**.
5. View the dashboard visualization, which shows:
    - **Disparity in performance** - how the selected performance metric compares for the subpopulations, including *underprediction* (false negatives) and *overprediction* (false positives).
    - **Disparity in predictions** - A comparison of the number of positive cases per subpopulation.
6. Edit the configuration to compare the predictions based on different performance metrics.

In [4]:
FairlearnDashboard(
    sensitive_features=S_test,
    sensitive_feature_names=['Age'],
    y_true=y_test,
    y_pred={'diabetes_model': diabetes_model.predict(X_test)},
)


FairlearnWidget(value={'true_y': [0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0…

The results show a much higher selection rate for patients over 50 than for younger patients. However, in reality, age is a genuine factor in diabetes, so you would expect more positive cases among older patients.

If we base model performance on *accuracy* (in other words, the percentage of predictions the model gets right), then it seems to work more or less equally for both subpopulations. However, based on the *precision* and *recall* metrics, the model tends to perform better for patients who are over 50 years old.

Let's see what happens if we exclude the **Age** feature when training the model.

In [5]:
ageless = features.copy()
ageless.remove('Age')
X2 = data[ageless].to_numpy()
y2 = data['Diabetic'].to_numpy()

X_train2, X_test2, y_train2, y_test2, S_train2, S_test2 = (
    model_selection.train_test_split(
        X2, y2, S, test_size=0.20, random_state=0, stratify=y2
    )
)

print("Training model...")
ageless_model = DecisionTreeClassifier().fit(X_train2, y_train2)
print("Model trained.")

FairlearnDashboard(
    sensitive_features=S_test2,
    sensitive_feature_names=['Age'],
    y_true=y_test2,
    y_pred=ageless_model.predict(X_test2)
)


Training model...
Model trained.


FairlearnWidget(value={'true_y': [0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0…

Explore the model in the dashboard.

When you review *recall*, note that the disparity has reduced, but the overall recall has also reduced because the model now significantly underpredicts positive cases for older patients. Even though **Age** was not a feature used in training, the model still exhibits some disparity in how well it predicts for older and younger patients.

In this scenario, simply removing the **Age** feature slightly reduces the disparity in *recall*, but increases the disparity in *precision* and *accuracy*. This underlines one the key difficulties in applying fairness to machine learning models - you must be clear about what *fairness* means in a particular context, and optimize for that.

## Register the model and upload the dashboard data to Azure Machine Learning

You've trained the model and reviewed the dashboard locally in this notebook; but it might be useful to register the model in your Azure Machine Learning workspace and create an experiment to record the dashboard data so you can track and share your fairness analysis.

Let's start by registering the original model (which included **Age** as a feature).

> **Note**: If prompted, follow the link and enter the authentication code provided to sign into your Azure subscription.

In [10]:
ws = Workspace.from_config()
print('Ready to work with', ws.name)


Ready to work with workspace


In [6]:
model_file = 'diabetes_model.pkl'
joblib.dump(diabetes_model, model_file)

print('Registering model...')
registered_model = Model.register(ws, model_file, 'diabetes_classifier')
model_id = registered_model.id

print('Model registered.', model_id)


Ready to work with workspace
Registering model...
Registering model diabetes_classifier
Model registered. diabetes_classifier:2


Now you can use the FairLearn package to create binary classification group metric sets for one or more models, and use an Azure Machine Learning experiment to upload the metrics.

> **Note**: This may take a while. When the experiment has completed, the dashboard data will be downloaded and displayed to verify that it was uploaded successfully.

In [8]:

sf = {'Age': S_test.Age}
ys_pred = {model_id: diabetes_model.predict(X_test)}
dash_dict = _group_metric_set._create_group_metric_set(
    y_test, ys_pred, sf, 'binary_classification'
)

exp = Experiment(ws, 'Diabetes_Fairness')
print(exp)

run = exp.start_logging()
try:
    dashboard_title = "Fairness insights of Diabetes Classifier"
    upload_id = fairness.upload_dashboard_dictionary(
        run, dash_dict, dashboard_title
    )
    print(f'\nUploaded to id: {upload_id}\n')

finally:
    run.complete()


Experiment(Name: Diabetes_Fairness,
Workspace: workspace)

Uploaded to id: cb82a935-6c01-4c41-8ffe-5cb4aed58dec

{'schemaType': 'dashboardDictionary', 'schemaVersion': 0, 'predictionType': 'binaryClassification', 'trueY': [0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 

The preceding code downloaded the metrics generated in the experiement just to confirm it completed successfully. The real benefit of uploading the metrics to an experiement is that you can now view the FairLearn dashboard in Azure Machine Learning studio.

Run the cell below to see the experiment details, and click the link to see the run in Azure Machine Learning studio. Then view the **Fairness** tab of the experiment run to view the dashboard, which behaves the same way as the widget you viewed previously in this notebook.

In [10]:
RunDetails(run).show()


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

You can also find the fairness dashboard by selecting a model in the **Models** page of Azure Machine Learning studio and reviewing its **Fairness** tab. This enables your organization to maintain a log of fairness analysis for the models you train and register.

## Mitigate unfairness in the model

Now that you've analyzed the model for fairness, you can use any of the *mitigation* techniques supported by the FairLearn package to find a model that achieves the best balance of predictive performance and fairness.

In this exercise, you'll use the **GridSearch** feature, which trains multiple models in an attempt to minimize the disparity of predictive performance for the sensitive features in the dataset (in this case, the age groups). You'll optimize the models by applying the **EqualizedOdds** parity constraint, which tries to ensure that models that exhibit similar true and false positive rates for each sensitive feature grouping. 

> *This may take some time to run*

In [7]:
print('Finding mitigated models...')

sweep = GridSearch(DecisionTreeClassifier(), EqualizedOdds())

sweep.fit(X_train, y_train, sensitive_features=S_train.Age)
models = sweep._predictors

model_dir = 'mitigated-models'
os.makedirs(model_dir, exist_ok=True)
model_name = 'diabetes_unmitigated'
print(model_name)
joblib.dump(diabetes_model, os.path.join(model_dir, f'{model_name}.pkl'))
predictions = {model_name: diabetes_model.predict(X_test)}
i = 0
for model in models:
    i += 1
    model_name = f'diabetes_mitigated_{i}'
    print(model_name)
    joblib.dump(model, os.path.join(model_dir, f'{model_name}.pkl'))
    predictions[model_name] = model.predict(X_test)


Finding mitigated models...
diabetes_unmitigated
diabetes_mitigated_1
diabetes_mitigated_2
diabetes_mitigated_3
diabetes_mitigated_4
diabetes_mitigated_5
diabetes_mitigated_6
diabetes_mitigated_7
diabetes_mitigated_8
diabetes_mitigated_9
diabetes_mitigated_10


Now you can use the FairLearn dashboard to compare the mitigated models:

Run the following cell and then use the wizard to visualize **Age** by **Recall**.

In [8]:
FairlearnDashboard(
    sensitive_features=S_test, 
    sensitive_feature_names=['Age'],
    y_true=y_test,
    y_pred=predictions
)

FairlearnWidget(value={'true_y': [0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0…

The models are shown on a scatter plot. You can compare the models by measuring the disparity in predictions (in other words, the selection rate) or the disparity in the selected performance metric (in this case, *recall*). In this scenario, we expect disparity in selection rates (because we know that age *is* a factor in diabetes, with more positive cases in the older age group). What we're interested in is the disparity in predictive performance, so select the option to measure **Disparity in recall**.

The chart shows clusters of models with the overall *recall* metric on the X axis, and the disparity in recall on the Y axis. Therefore, the ideal model (with high recall and low disparity) would be at the bottom right corner of the plot. You can choose the right balance of predictive performance and fairness for your particular needs, and select an appropriate model to see its details.

An important point to reinforce is that applying fairness to a model is a trade-off between overall predictive performance and disparity across sensitive feature groups - generally you must sacrifice some overall predictive performance to ensure that the model predicts fairly for all segments of the population.

> **Note**: Viewing the *precision* metric may result in a warning that precision is being set to 0.0 due to no predicted samples - you can ignore this.

## Upload the mitigation dashboard metrics to Azure Machine Learning

As before, you might want to keep track of your mitigation experimentation. To do this, you can:

1. Register the models found by the GridSearch process.
2. Compute the performance and disparity metrics for the models.
3. Upload the metrics in an Azure Machine Learning experiment.

In [15]:
registered_model_predictions = {}
for model_name, prediction_data in predictions.items():
    model_file = os.path.join(model_dir, model_name + ".pkl")
    registered_model = Model.register(ws, model_file, model_name)
    registered_model_predictions[registered_model.id] = prediction_data

sf = {'Age': S_test.Age}
dash_dict = _group_metric_set._create_group_metric_set(
    y_test, registered_model_predictions, sf, 'binary_classification'
)

exp = Experiment(ws, 'Diabetes_Fairness_Mitigation')
print(exp)

run = exp.start_logging()
RunDetails(run).show()
try:
    dashboard_title = 'Fairness Comparison of Diabetes Models'
    upload_id = fairness.upload_dashboard_dictionary(
        run, dash_dict, dashboard_title
    )
    print(f'\nUploaded to id: {upload_id}\n')
finally:
    run.complete()


Registering model diabetes_unmitigated
Registering model diabetes_mitigated_1
Registering model diabetes_mitigated_2
Registering model diabetes_mitigated_3
Registering model diabetes_mitigated_4
Registering model diabetes_mitigated_5
Registering model diabetes_mitigated_6
Registering model diabetes_mitigated_7
Registering model diabetes_mitigated_8
Registering model diabetes_mitigated_9
Registering model diabetes_mitigated_10


/anaconda/envs/fair/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Experiment(Name: Diabetes_Fairness_Mitigation,
Workspace: workspace)


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…


Uploaded to id: 97b910c9-8402-468b-b100-3f9182b03894



> **Note**: A warning that precision is being set to 0.0 due to no predicted samples may be displayed - you can ignore this.


When the experiement has finished running, click the link in the widget at the bottom of the output to view the run in Azure Machine Learning studio, and view the FairLearn dashboard on the **fairness** tab.